pip3 install a-world-of-countries

In [17]:
import pandas as pd
import numpy as np
import awoc
from sklearn import preprocessing
import sklearn

In [18]:
pd.__version__

'1.5.3'

In [19]:
np.__version__

'1.23.5'

In [20]:
sklearn.__version__

'1.2.2'

In [21]:
df = pd.read_csv("LifeExpectancyData.csv")

In [22]:
df.columns

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')

In [23]:
df.head(5)

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [24]:
df["Country"].nunique(), min(df["Year"].to_list()), max(df["Year"].to_list())

(193, 2000, 2015)

In [25]:
len(df), len(df.columns)

(2938, 22)

In [26]:
df[df.isna().any(axis=1)].head(5)

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
32,Algeria,2015,Developing,75.6,19.0,21,NaN,0.000000,95.0,63,...,95.0,NaN,95.0,0.1,4132.762920,39871528.0,6.0,5.8,0.743,14.4
44,Algeria,2003,Developing,71.7,146.0,20,0.34,25.018523,NaN,15374,...,87.0,3.60,87.0,0.1,294.335560,3243514.0,6.3,6.1,0.663,11.5
45,Algeria,2002,Developing,71.6,145.0,20,0.36,148.511984,NaN,5862,...,86.0,3.73,86.0,0.1,1774.336730,3199546.0,6.3,6.2,0.653,11.1
46,Algeria,2001,Developing,71.4,145.0,20,0.23,147.986071,NaN,2686,...,89.0,3.84,89.0,0.1,1732.857979,31592153.0,6.4,6.3,0.644,10.9
47,Algeria,2000,Developing,71.3,145.0,21,0.25,154.455944,NaN,0,...,86.0,3.49,86.0,0.1,1757.177970,3118366.0,6.5,6.4,0.636,10.7


In [27]:
df.dtypes

Country                             object
Year                                 int64
Status                              object
Life expectancy                    float64
Adult Mortality                    float64
infant deaths                        int64
Alcohol                            float64
percentage expenditure             float64
Hepatitis B                        float64
Measles                              int64
 BMI                               float64
under-five deaths                    int64
Polio                              float64
Total expenditure                  float64
Diphtheria                         float64
 HIV/AIDS                          float64
GDP                                float64
Population                         float64
 thinness  1-19 years              float64
 thinness 5-9 years                float64
Income composition of resources    float64
Schooling                          float64
dtype: object

In [28]:
# df = df.dropna()
# df.to_csv("normalizedData-RemovedNanRows.csv")
# df

This removes about 50% of about data - we need to be more careful about removing these nans...

Fill in Nan values based on this approach: 

If there is other values from the same country, we take the mean of those values and fill in empty data. (if there is none, we use mean)

If over 200 nans in one column, we remove as they have way too many nans

In [29]:
start_nans = df.isna().sum().sum()
df_without_nans = df.copy()
df_without_nans.drop(["Hepatitis B", "GDP", "Population"], axis="columns", inplace=True)
columns_to_correct_nans = list(df_without_nans.columns)[3:]
print("Number of nans before this procedure: ", start_nans)
for index, row in df_without_nans.iterrows():
    for column in columns_to_correct_nans:
        if pd.isna(row[column]):
            current_country = row['Country']
            same_country_rows = df_without_nans[df_without_nans['Country'] == current_country]
            mean_value = same_country_rows[column].mean()
            df_without_nans.at[index, column] = mean_value

print("Filled with mean from rows with same country: ", start_nans - df_without_nans.isna().sum().sum())
start_nans = df_without_nans.isna().sum().sum()
df_without_nans.fillna(df_without_nans.mean(), inplace=True)
print("Filled with mean from all rows: ", start_nans - df_without_nans.isna().sum().sum())

Number of nans before this procedure:  2563
Filled with mean from rows with same country:  2062
Filled with mean from all rows:  501


/var/folders/pl/7f1cr2657p3bnpdt3bnm_9w00000gn/T/ipykernel_60547/2741766383.py:16: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_without_nans.fillna(df_without_nans.mean(), inplace=True)


Here we do normalization.

We could maybe also do min-max normalization to get values between 0 and 1

In [30]:
columns_to_standardize = list(df_without_nans.columns)[3:]
for column in columns_to_standardize:
    mean = df_without_nans[column].mean()
    std_dev = df_without_nans[column].std()
    df_without_nans[column] = (df_without_nans[column] - mean) / std_dev

In [31]:
df_without_nans.shape, df_without_nans.isna().sum().sum()

((2938, 19), 0)

In [32]:
df_without_nans.head(10)

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,-0.444372,0.791452,0.268778,-1.139938,-0.335513,-0.110366,-0.964550,0.255316,-3.254219,0.911749,-0.723742,-0.32339,2.812651,2.772807,-0.725277,-0.579832
1,Afghanistan,2014,Developing,-0.980783,0.855926,0.285738,-1.139938,-0.334384,-0.168095,-0.989641,0.274014,-1.039197,0.919893,-0.850094,-0.32339,2.880918,2.817423,-0.739924,-0.610466
2,Afghanistan,2013,Developing,-0.980783,0.831748,0.302697,-1.139938,-0.334537,-0.173502,-1.014732,0.292712,-0.868811,0.899533,-0.765859,-0.32339,2.926429,2.862039,-0.769218,-0.641099
3,Afghanistan,2012,Developing,-1.022854,0.863985,0.328137,-1.139938,-0.332040,0.032040,-1.039823,0.317642,-0.655828,1.058344,-0.639507,-0.32339,2.971940,2.928963,-0.803395,-0.671733
4,Afghanistan,2011,Developing,-1.054408,0.888163,0.345097,-1.139938,-0.367800,0.051748,-1.059895,0.342573,-0.613232,0.793659,-0.597390,-0.32339,3.040207,2.973578,-0.847336,-0.763634
5,Afghanistan,2010,Developing,-1.096479,0.920400,0.370536,-1.139938,-0.331288,-0.037550,-1.084986,0.373736,-0.698425,1.335244,-0.681624,-0.32339,3.085718,3.018194,-0.876630,-0.855535
6,Afghanistan,2009,Developing,-1.117515,0.936519,0.395976,-1.139938,-0.342816,0.038493,-1.110077,0.398666,-0.826214,1.424830,-0.807976,-0.32339,3.131229,3.085118,-0.944982,-0.947436
7,Afghanistan,2008,Developing,-1.170104,0.984875,0.421415,-1.134972,-0.358354,-0.071559,-1.135167,0.423597,-0.783618,0.980974,-0.765859,-0.32339,3.176740,3.129734,-0.949865,-1.008703
8,Afghanistan,2007,Developing,-1.233212,1.049349,0.438375,-1.137455,-0.365881,-0.111499,-1.160258,0.442295,-0.826214,0.329444,-0.807976,-0.32339,3.222251,3.174350,-1.037747,-1.100605
9,Afghanistan,2006,Developing,-1.254247,1.049349,0.455335,-1.134972,-0.362732,-0.037462,-1.185349,0.460993,-1.039197,0.614488,-1.018563,-0.32339,3.267762,3.218966,-1.086570,-1.192506


In [33]:
df_without_nans.to_csv("StandardizedDataFrameWithNansFilled.csv")